# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
#HK1: Obtained all dependencies from second Assignment

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import  TabularDatasetFactory

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#HK2: Get the workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-final-project'

experiment=Experiment(ws, experiment_name)


In [5]:
#HK3: Get the Cluster

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "assign3-cluster1"
amlcompute_cluster_name = "notebook194794"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Found existing cluster, use it.


In [39]:
data = pd.read_csv("./heart.csv")
df = pd.DataFrame(data)

if not os.path.isdir('data'):
    os.mkdir('data')

df.to_csv("data/heart.csv", index=False)
ds = ws.get_default_datastore()
ds.upload(src_dir='./data',overwrite=True, show_progress=True)
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('heart.csv'))
train_data = train_data.register(workspace=ws,name='heart',description='test')


Uploading an estimated of 1 files
Uploading ./data/heart.csv
Uploaded ./data/heart.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [40]:
#data_location = "https://github.com/AI-Treasure/Capstone-3rd-assignment/heart.csv"
#data_location = "./heart.csv"
#ds = TabularDatasetFactory.from_delimited_files(path=data_location)

In [41]:
df.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [43]:
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    max_concurrent_iterations=4,
    enable_early_stopping=True,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='HeartDisease',
    n_cross_validations=2)

In [ ]:
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
#automl_config = AutoMLConfig()

In [44]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# HK-Step 11: Retrieve and save your best automl model.

best_run_auto, best_fit_model = remote_run.get_output()

print(best_run_auto)
print(fitted_model)

print("best-run details :", best_run_auto.get_details())
print('\n')
print("best-run id : ",best_run_auto.id)
print('\n')
print("best-run file names :", best_run_auto.get_file_names())
print('\n')
print("best-run metrics :", best_run_auto.get_metrics())

best_run_auto.register_model(model_name = 'best_automl_register', model_path = '/outputs/')

In [ ]:
#TODO: Save the best model


joblib.dump(best_fit_model, 'best_fit_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#MAAK NOG NETJES (Heb het model eigenlijk al geregistreerd)

from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

In [ ]:
#Maak nog netjes
# Deploy the Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#PAS ~DIT NOG AAN

import requests
import json

# URL for the web service
scoring_uri = 'http://cf4e6ab2-17dc-4a7e-9c00-6d04b9e8a24c.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
                "gender_Male" :0,
                "SeniorCitizen":0,
                "Dependents" :0,
                "Partner" :0,
                "tenure":2,
                "PhoneService" :1,
                "MultipleLines" :1,
                "InternetService_Fiber optic" :1,
                "InternetService_No":0,
                "OnlineSecurity" :0,
                "OnlineBackup" :0,
                "DeviceProtection" :0,
                "TechSupport" :0,
                "StreamingTV" :0,
                "StreamingMovies" :1,
                "Contract_One year" : 0,
                "Contract_Two year":0,
                "PaperlessBilling" :1,
                "PaymentMethod_Bank transfer (automatic)":0,
                "PaymentMethod_Credit card (automatic)":0,
                "PaymentMethod_Mailed check":0,
                "PaymentMethod_Electronic check" :1,
                "MonthlyCharges" : 85.7,
                "TotalCharges" : 169.8 }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
